In [13]:
import os
import pandas as pd
from tqdm.notebook import tqdm # while using notebook
# from tqdm import tqdm # if/when changing for .py script
import cv2
import skimage.io

def select_directory(msg):
    user_input = input(msg)
    if not os.path.exists(user_input):
        print("Selected path not found.")
        user_input = select_directory(msg)
    return user_input

# --- Support variables ---
ALPHABET = 'abcdefghijklmnopqrstuvwxyz'.upper()


In [14]:
# --- Select the input and output directories ---
input_directory:str = select_directory("Select the images directory path.")
output_directory:str = select_directory("Select the output directory path.")

In [ ]:
# --- Read the archive folder ---
file_list = os.listdir(input_directory) #remove the suffix.
file_list = [file for file in file_list if file.endswith(".tiff")]
file_list.sort()

# Start defining the dataframe
df = pd.DataFrame(columns=['Original_name', 'Date' ,'Timestamp', 'Channel ID', 'Well ID']) # IMPORTANT: This line might need to be changed depending on the file name structure.
for row in range(len(file_list)):   
    df.loc[row] = [file_list[row]] + file_list[row].replace(".tiff","").split("-")
df["Well Name"] = df["Well ID"].map(lambda well_id: ALPHABET[int(well_id[0:2])-1] + well_id[2:4])

# --- Copy files into the new directory ---
for i in tqdm(range(len(file_list))): # while using notebook
    # if i != 10: continue # This is used for unit testing.
    
    # Define the name structure
    output_name = f'{df.loc[i,"Well Name"]}_{df.loc[i,"Channel ID"]}.jpg' # You control the output format here.
    src_dir = os.path.join(input_directory,df.loc[i,"Original_name"])
    output_dir = os.path.join(
        output_directory,
        output_name
    )
    
    # Check if the file already exists
    if os.path.exists(output_dir): FileExistsError(f"{output_name} already exists.")
    
    # Read the image
    img = skimage.io.imread(src_dir)
    
    # Rescale the image
    img = skimage.exposure.rescale_intensity(img)
    img = cv2.convertScaleAbs(img, alpha=(255.0/65535.0))    
    
    # Save the image
    skimage.io.imsave(output_dir, img)